# Similarity with fasttext

### Load model

In [10]:
import csv

import fasttext
import fasttext.util
import pandas as pd
#fasttext.util.download_model('nl', if_exists='ignore')  # Dutch

In [2]:
ft = fasttext.load_model('cc.nl.300.bin')

### Load train and test df

In [89]:
# Train
train = pd.read_csv("result.csv")
cleanup_label = {"sentiment": {"Ja": 1, "Nee": 0}}
train.replace(cleanup_label, inplace=True)
# Drop duplicate label
train.drop(["label"], axis=1, inplace=True)
train.rename(columns={"sentiment":"label"}, inplace=True)

# Test
test = pd.read_csv("df_to_label.csv")
#test.rename(columns={"label":"sentiment"}, inplace=True)
test["label"] = None
test = test.loc[~test.index.isin(train.index)]

In [93]:
train['label'] = ['__label__'+str(s) for s in train["label"]]

In [95]:
dff = train[["label", "text_wop_clean"]]
dff.to_csv(r'merge.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

### Train model

In [99]:
model = fasttext.train_supervised(input="merge.txt", lr=0.0001, epoch=5000, wordNgrams=2, bucket=200000, dim=400, loss='hs')

### Predict

In [97]:
test

,Unnamed: 0,index,index_articles,type,text,article_name,date,index_article,article_filepath,dir,...,newspaper_city,newspaper_publisher,newspaper_source,newspaper_volume,newspaper_issuenumber,newspaper_language,count,text_wop,text_wop_clean,label
68,68,552290,122121,p,ROTTERDAM — Een grensconflict' tussen Nederlan...,DDD_010963472_0158_articletext.xml,1990-03-13,2739936,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,...,Rotterdam,De Arbeiderspers,Gemeentearchief Rotterdam,45.0,13123.0,nl,15,rotterdam — een grensconflict tussen nederland...,grensconflict tussen oorzaak kapitein gezagvoe...,None
69,69,735331,33576,p,PkJ^.Van Huis Uit Aktief? BRUNSSUM: Goed onder...,DDD_010623751_0362_articletext.xml,1990-02-10,3061323,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,...,Heerlen,Nieuwe Limburger koerier;Uitgeversmaatschappĳ ...,KB C 101,72.0,35.0,nl,15,pkjvan huis uit aktief brunssum goed onderhoud...,huis goed onderhouden aantrekkelijk woonhuis g...,None
70,70,735332,23736,p,O. TIEKSTRA T. TIEKSTRA-TEN HAVE Rein en Frouw...,DDD_010559856_0107_articletext.xml,1991-12-31,3358244,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,...,Amersfoort,Stichting Nederlands Dagblad,Nederlands Dagblad,48.0,11974.0,nl,15,o tiekstra t tiekstraten have rein en frouwina...,have rein kievit til jan veen ha ga veen veend...,None
71,71,2,13967,p,"NAALDWIJK 20 Dec. — Alicantcn 1.10—1.37, stoof...",DDD_110585231_0104_articletext.xml,1950-12-21,2055316,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,...,Amsterdam,Dagblad De Telegraaf,KB C 98,53.0,19710.0,nl,15,naaldwijk 20 dec — alicantcn 110—137 stoofpere...,stoofperen andijvie kool boerenkool spruiten w...,None
72,72,552289,71208,p,optie serie ■ omzet vk sk ond.w AEGN c okt 90 ...,DDD_011019405_0362_articletext.xml,1990-08-04,3172467,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,...,Groningen,Nieuwenhuis,Groninger archieven,103.0,182.0,nl,15,optie serie ■ omzet vk sk ondw aegn c okt 90 1...,optie serie omzet jan po jan pokt pokt pokt po...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,891361,25677,p,"BRUNSSUM: Ruim vrijst woonhuis met c.v.-gas, d...",DDD_010623940_0366_articletext.xml,1990-11-24,2907662,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,...,Heerlen,Nieuwe Limburger koerier;Uitgeversmaatschappĳ ...,KB C 101,72.0,279.0,nl,8,brunssum ruim vrijst woonhuis met cvgas dubbel...,ruim vrijst woonhuis dubbele garage mooie tuin...,None
996,996,891362,49914,p,""" Druk PRIVE-HUIS vr. v. dir. 5 enkele Holl. m...",DDD_010646021_0320_articletext.xml,1990-01-19,2565783,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,...,Amsterdam,Dagblad De Telegraaf,KB C 98,98.0,31727.0,nl,8,druk privehuis vr v dir 5 enkele holl meisjes...,druk enkele meisjes zeer hoge meisjes chauffeu...,None
997,997,891364,144313,p,(advertentie) Als je vol tankt en je ziet dat ...,DDD_010691611_0078_articletext.xml,1993-09-15,197914,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,...,Amsterdam,Dagblad De Telegraaf,KB C 98,101.0,32852.0,nl,8,advertentie als je vol tankt en je ziet dat je...,advertentie vol tankt ziet piek kwijt bent wor...,None
998,998,891378,110113,p,Omdat wij al 20 jaar onze gastvrouwen en onze ...,DDD_010691719_0351_articletext.xml,1993-01-21,186596,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,...,Amsterdam,Dagblad De Telegraaf,KB C 98,101.0,32652.0,nl,8,omdat wij al 20 jaar onze gastvrouwen en onze ...,wij jaar onze gastvrouwen onze uiterst zorgvul...,None


In [104]:
print(test["text"][73])
model.predict(test["text_wop_clean"][73])
#test.iloc[7,1]#.values

GENè VE - Aanstaande woensdag komen de ministers van olie van de OPEC (Organisatie van olieexporterende landen) in Genève bij elkaar om te proberen de olieprijzen omhoog te halen. Ze zien zich nog steeds geconfronteerd met een te groot aanbod van olie op de wereldmarkt, met als gevolg olieprijzen die ver onder hun streefprijs van 21 dollar per vat (159 liter) liggen. De gemiddelde prijs van de diverse oliesoorten van de Opec ligt op het ogenblik op 16,95 dollar per vat. De enige manier om de prijs op het gewenste niveau te krijgen is een flinke vermindering van de Opec-olieproduktie. Het officiële produktiemaximum van de olieorganisatie staat op 23,65 miljoen vaten olie per dag, maar in december werden er 24,2 miljoen vaten olie per dag geproduceerd. De New Vork Times meldde maandag dat de voorzitter van de Opec en tevens minister van olie van Nigeria, Jibril Aminu, tijdens de Opec-vergadering zal aandringen op een vermindering van de olieproduktie met zes procent ofwel ongeveer 1,5 mi

(('__label__0',), array([0.50001067]))